# Predicting Baklava Cravings: A Sweet Data Science Problem 🍭

Did you know that Istanbul alone devours a staggering 2,000 tons of baklava during the festive season? That's a lot of sweet treats!

To ensure that bakeries and retailers can meet this incredible demand without overproducing or understocking, we can leverage the power of data science. By analyzing historical sales data, consumer trends, and external factors like holidays, we can build a predictive model to forecast baklava demand with impressive accuracy.

You are asked to experiment with different the hyperparameters, and store your experiments in the Model Registry to compare the results in order to deploy the best performing one!

# 🐠 Install & Import packages
We will need to install and import packages as we develop our model.

This will take a couple of minutes, and if pip gives any error, don't worry about it. Things will just run fine regardless.

In [ ]:
!pip -q install keras "tensorflow==2.15.1" "tf2onnx" "onnx" "seaborn" "onnxruntime" model-registry=="0.2.9"

In [ ]:
from pathlib import Path
import pickle
import os
import logging, warnings

# Suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import tf2onnx
import onnx
import tensorflow as tf
import joblib

from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
from matplotlib import pyplot as plt
import onnxruntime as ort


# Suppress CUDA and TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=FutureWarning)

# 📦 Load Data
Let's load our dataset, that consist of baklava consumption information alongside the number of bakeries for the past 10 years in different regions of Türkiye.

Then we will select the input and output data.

In [ ]:
# Load the data
data = pd.read_csv("synthetic_baklava_data_turkey.csv")

In [ ]:
# Feature Engineering
data['Population_per_Bakery'] = data['Population'] / data['Bakery_Count'] 

In [ ]:
# One-hot encode the 'Region' column
region_encoder = OneHotEncoder(sparse_output=False)
region_encoded = region_encoder.fit_transform(data[["Region"]])
region_encoded_df = pd.DataFrame(region_encoded, columns=region_encoder.get_feature_names_out(["Region"]))
data = pd.concat([data, region_encoded_df], axis=1)

In [ ]:
# Get list of regions
regions = list(data.columns[data.columns.str.startswith('Region_')])


Input data (X) contains baklava consumption per region in the country in each day, with a detail whether it was a holiday season or not.

Output data (y) is the target variable the model is trying to predict. In this case, y is the 'Demand' column which represents the demand for the upcoming holiday season. The model will learn to predict the demand based on the previous consumptions.

In [ ]:
# Prepare data for modeling
features = ['Holiday_Promotion', 'Population_per_Bakery', 'Income_Level', 'Holiday_Season'] + regions
X = data[features] 
y = data['Demand']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🚀 Build the model

This is where we need your help! Experimentation and exploration are key to finding the best settings for our specific dataset and problem. 

Below 4 little parameteres that are called as hyperparameters and we need your help to define the best settings for the model!

It has some good starting values, but they can be better for sure!

In [ ]:
# Epochs represents the number of times the model sees the entire training dataset. 
# Higher values can improve accuracy but may also lead to overfitting.
epochs = 100  

# batch_size is the number of samples processed before the model updates its internal parameters. 
# Larger batch sizes can speed up training but may require more memory.
batch_size = 32  

# learning_rate controls the step size during weight updates. 
# Higher values can lead to faster convergence but may cause instability if too high.
learning_rate = 0.001  

# Number of neurons in the hidden layer. For simpler problems, fewer neurons may suffice. 
# For more complex problems, a larger number of neurons may be necessary.
# value can be 16, 32, 64..
hidden_layer_units = 16

The below piece of code is the model definition. It will uses the parameters you define up there and learned to predict the demand as accurately as possible.

Then, we check how well the model is doing at making the guesses!

In [ ]:
def train_and_evaluate_nn():
    """
    Train and evaluate a simple neural network model.

    Args:
        X_train_scaled: Scaled training features.
        y_train: Training target values.
        X_test_scaled: Scaled test features.
        y_test: Test target values.
        hidden_layer_units: Number of units in hidden layer. Defaults to 64.
        learning_rate: Learning rate for optimizer. Defaults to 0.001.
        epochs: Number of training epochs. Defaults to 100.
        batch_size: Batch size for training. Defaults to 32.

    Returns:
        Tuple containing:
            - trained Keras Sequential model
            - Mean Squared Error (MSE) on test set
            - Mean Absolute Error (MAE) on test set
    """
    # Define the model architecture
    model = Sequential()
    model.add(
        Dense(
            hidden_layer_units,
            activation='relu',
            input_shape=(X_train_scaled.shape[1],)
        )
    )
    model.add(Dense(1))

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Train the model
    history = model.fit(
        X_train_scaled,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    # Make predictions
    y_pred = model.predict(X_test_scaled)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Print training parameters and metrics
    print(f"Epochs: {epochs}")
    print(f"Batch Size: {batch_size}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Hidden Layer Units: {hidden_layer_units}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")

    # Plot training history
    plt.plot(history.history['loss'])
    plt.title('Model Loss During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.show()

    return model, mse, mae

# 🏃 Train & Evaluate the Model

Let's kick of the training then! You'll get a nice plot and 2 important metrics to decide how well your parameters did.


### Interpretation of the Loss Plot
**X-axis:** Represents the number of training epochs.

**Y-axis:** Represents the training loss, which measures how well the model is fitting the training data. A lower loss generally indicates better performance.

**Decreasing Loss:** Ideally, the plot should show a downward trend, indicating that the model is learning and improving.

**Plateau/Increase:** If the loss plateaus or starts to increase, it might suggest overfitting or an inappropriate learning rate.

Experimentation and exploration are key to finding the best hyperparameter values for your specific dataset and problem.

In [ ]:
# Train and evaluate the model
model, mse, mae = train_and_evaluate_nn() 

# 🫡 Save the Model

Here we convert our trained prediction model into a popular format called ONNX so we can serve it from OpenShift AI.

### 🚨🚨 please make sure you update the below cell 🚨🚨

In [ ]:
# 🚨🚨 Specify a version - use your name to make it unique 🚨🚨
# example
# version = "user1-0.0.1"
version = "<YOUR_USER_NAME>-<VERSION>"

In [ ]:

# Create the artifacts directory if it doesn't exist
artifact_path = f"models/{version}/baklava/1/artifacts"
Path(artifact_path).mkdir(parents=True, exist_ok=True)

# Save the model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(
    model, 
    input_signature=[tf.TensorSpec([None, X_train_scaled.shape[1]], tf.float32, name='input')]
)
onnx.save(onnx_model, f"models/{version}/baklava/1/model.onnx")

# Save the scaler
joblib.dump(scaler, f"models/{version}/baklava/1/artifacts/scaler.pkl")

# 🪣 Upload your model to the bucket for (possible) deployment
Let's upload it to MinIO `models` bucket _in case this is the best performing one_.

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [ ]:
local_models_directory = f"models/{version}/baklava"

if not os.path.isdir(local_models_directory):
    raise ValueError(f"The directory '{local_models_directory}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")

num_files = upload_directory_to_s3("models", "models")

if num_files == 0:
    raise ValueError("No files uploaded.  Did you finish training and "
                     "saving the model to the \"models\" directory?  "
                     f"Check for \"models/{version}/baklava/1/model.onnx\"")

# 📫 Store Your Experiment and Deploy the BEST one

Now, it's time to save this experiment! We need you to store below information in Model Registry for each of your experiment. Go back to OpenShift AI Dashboard. On the left hand side, you'll see a `Model Registry` created for you (and for everybody else joining this effort). Register your model by providing the below information: 

- Model Name and Version
- Model Location (Path)
- Number of Epochs
- Batch Size
- Learning Rate
- Hidden Layer Units
- Mean Squared Error (MSE)
- Mean Absolute Error (MAE)

Then on to the new experiment! (3 times is good enough I'd say:))

print(f"Model Name: baklava")
print(f"Model Version: {version}")
print(f"Source model format: onnx")
print(f"Source model format version: 1")
print(f"Model Path: models/{version}/baklava/")
print(f"Epochs: {epochs}")
print(f"Batch Size: {batch_size}")
print(f"Learning Rate: {learning_rate}")
print(f"Hidden Layer Units: {hidden_layer_units}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")